In [1]:
import gzip
from random import shuffle

def readGz(f):
    for l in gzip.open(f):
        yield eval(l)

from collections import defaultdict
import numpy as np
data = []
count = 0;

for l in readGz("train.json.gz"):
    data.append(l)
    
shuffle(data) #shuffling data

trainData, validData = np.array_split(data, 2)

#trainData, validData = data, data[150000:200000]

businessCount = defaultdict(int)
visitedBusinesses = defaultdict(int)
users = defaultdict(int)
totalVisits = 0;

for d in trainData:
    user, business = d['userID'], d['businessID']
    businessCount[business] += 1
    totalVisits += 1
    visitedBusinesses[user+'-'+business] += 1
    users[user] += 1
    
businessList = []
usersList = []

vbusinessCount = defaultdict(int)
vvisitedBusinesses = defaultdict(int)
vusers = defaultdict(int)
vtotalVisits = 0;
vnotvisited = defaultdict(int)

for d in validData:
    user, business = d['userID'], d['businessID']
    vbusinessCount[business] += 1
    vtotalVisits += 1
    vvisitedBusinesses[user+'-'+business] += 1
    vusers[user] += 1

for c in vbusinessCount:
    businessList.append(c)
for u in vusers:
    usersList.append(u)

import random

while(len(vnotvisited)<100000):
    pair = random.choice(usersList)+'-'+random.choice(businessList)
    while(pair in vvisitedBusinesses):
        pair = random.choice(usersList)+'-'+random.choice(businessList)
    vnotvisited[pair] += 1

In [2]:
'''for u1 in users:
    for u2 in users:
        if(u1==u2):
            userCosSimilarity[u1][u2] = 1;
        else:
            count1 = 0
            count2 = 0
            combinedCount = 0
            for b in businessCount:
                u1b = visitedBusinesses[u1+'-'+b]
                u2b = visitedBusinesses[u2+'-'+b]
                if(u1b>0 and u2b>0):
                    count1+=1
                    count2+=1
                    combinedCount+=1
                elif(u1b==0 and u2b !=0):
                    count2+=1
                elif(u1b!=0 and u2b == 0):
                    count1+=1
            userCosSimilarity[u1][u2] = combinedCount/(math.sqrt(count1)*math.sqrt(count2))'''

busIndex = defaultdict(int)
userIndex = defaultdict(int)

indStart = 0
for b in businessCount:
    if b not in busIndex:
        busIndex[b] = indStart
        indStart += 1
print(indStart)
indStart = 0
for u in users:
    if u not in userIndex:
        userIndex[u] = indStart
        indStart += 1
print(indStart)
                

20544
18043


In [3]:
from scipy.sparse import csr_matrix
import sklearn.metrics

data = [1 for x in range(len(trainData))]
row = []
column = []
visitedCategory = defaultdict(set) # The categories visited by user
userVisited = defaultdict(list)
busVisitedBy = defaultdict(list)

for d in trainData:
    column.append(busIndex[d['businessID']])
    row.append(userIndex[d['userID']])
    for c in d['categories']:
        visitedCategory[d['userID']].add(c)
    userVisited[userIndex[d['userID']]].append(busIndex[d['businessID']])
    busVisitedBy[busIndex[d['businessID']]].append(userIndex[d['userID']])

uiMat = csr_matrix( (data,(row,column)), shape=(len(users),len(businessCount)) )
print(uiMat.shape)
userSimilarity = (sklearn.metrics.pairwise.cosine_similarity(uiMat, dense_output=False))
busSimilarity = (sklearn.metrics.pairwise.cosine_similarity(uiMat.transpose(), dense_output=False))

print(userSimilarity.shape)
userList = np.argsort(np.array(userSimilarity[0].todense()))[0][-10:]
print(userList)
print(userSimilarity[0, 1405])



(18043, 20544)
(18043, 18043)
[ 6000  5999  1851  7093  2823  5777  8335 10047 12907     0]
0.0


In [4]:
mostPopular = [(businessCount[x], x) for x in businessCount]
mostPopular.sort()
mostPopular.reverse()

return1 = set()
count = 0
for ic, i in mostPopular:
  count += ic
  return1.add(i)
  if count > 0.50*totalVisits: break
    
umostPopular = [(users[x], x) for x in users]
umostPopular.sort()
umostPopular.reverse()

ureturn1 = set()
count = 0
for ic, i in mostPopular:
  count += ic
  ureturn1.add(i)
  if count > 0.50*totalVisits: break

In [8]:
def predict(ubPairs, visited, k): # k is the k nearest neighbors we are considering
    global wPred
    global cPred
    global return1
    global return2
    for p in ubPairs:
        u,b = p.strip().split('-')
        match = False
        if u in userIndex and b in busIndex:
            uInd = userIndex[u]
            bInd = busIndex[b]
            #simUsers = np.argsort(np.array(userSimilarity[uInd].todense()))[0][-len(users):]
            visitedCount = 0
            sumSim = 0.0
            '''for su in simUsers:
                if(bInd in userVisited[su]):
                    visitedCount+=1'''
            for su in busVisitedBy[bInd]:
                sumSim += userSimilarity[uInd, su]
                visitedCount += 1
            if(sumSim > 0.1):
                match = True
        elif u not in userIndex and b in busIndex:
            if b in return1:
                match = True
        elif u in userIndex and b not in busIndex:
            if u in ureturn1:
                match = True
        if((match and visited) or (not match and not visited)):
            cPred += 1
        else:
            wPred += 1

In [9]:
cPred = 0
wPred = 0
predict(vvisitedBusinesses, True, 10)
predict(vnotvisited, False, 10)
print(wPred)
print(cPred)
print(wPred/(wPred+cPred))

65229
134771
0.326145


In [11]:
def categoryVisitedPredFile(inputFile, outputFile):
    predictions = open(outputFile, 'w')
    global wPred
    global cPred
    global return1
    global return2
    for l in open(inputFile):
        if l.startswith("userID"):
            predictions.write(l)
            continue
        u,b = l.strip().split('-')
        match = False
        if u in userIndex and b in busIndex:
            uInd = userIndex[u]
            bInd = busIndex[b]
            sumSim = 0.0
            count = 0
            busSim = 0.0
            for su in busVisitedBy[bInd]:
                sumSim += userSimilarity[uInd, su]
                count += 1
            sumSim = sumSim/count
            count = 0
            for bu in userVisited[uInd]:
                busSim += busSimilarity[bInd, bu]
                count += 1
            busSim = busSim/count
            #if(sumSim > 0.0 or busSim > 0.01):
            if(sumSim > 0.0):
                match = True
        elif u not in userIndex and b in busIndex:
            if b in return1:
                match = True
        elif u in userIndex and b not in busIndex:
            if u in ureturn1:
                match = True
        if(match):
            predictions.write(u + '-' + b + ",1\n")
        else:
            predictions.write(u + '-' + b + ",0\n")

categoryVisitedPredFile ("pairs_Visit.txt", "predictions_Visit.txt") 
print("done")

done
